# Spark Streaming 编程
---

## 概述

Spark Streaming是核心Spark API的扩展，可实现实时数据流的可扩展、高吞吐量、容错流处理。

Streaming框架如下：数据从多种数据源如Kafka、Kinesis、TCP套接字获取，经过高级别功能函数如`map`、`reduce`、`join`、`window`等表达的复杂算法处理,处理后的数据可推送到文件系统、数据库、实时仪表板。

![streaming-arch](../images/streaming-arch.png)

内部其工作方式如下：Spark Streaming接受实时输入数据流，并将数据流分批，然后由Spark引擎进行处理，最终生成批处理的结果流。

![streaming-flow](../images/streaming-flow.png)

Spark Streaming用称为离散流(DStream, discretized stream)的高级抽象来表示连续的数据流。DStream可根据来自Kafka和Kinesis等源的输入数据流来创建，也可通过对其他DStream应用高级操作来创建。在内部，DStream表示为RDD序列。

## Hello World

计算从侦听TCP套接字的数据服务器接收到的文本数据中的单词数。

In [1]:
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

# 创建具有两个执行线程和1秒批处理间隔的本地StreamingContext
sc = SparkContext("local[2]", "NetworkWordCount")
ssc = StreamingContext(sc, 1)

# 使用此上下文创建一个DStream，表示来自TCP源的流数据
lines = ssc.socketTextStream("localhost", 9999)

# DStream中每个记录都是一行文本，将其分成单子列表 
words = lines.flatMap(lambda line: line.split(" "))

pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKey(lambda x, y: x + y)

wordCounts.pprint()

在执行下列代码前，在另一终端开启netcat服务器 `nc -lk 9999` 并输入hello world，待启动下列代码后再回车。

In [5]:
# 设置好DStream启动计算
ssc.start()
import time
time.sleep(5)
ssc.stop()

-------------------------------------------
Time: 2020-09-15 13:19:12
-------------------------------------------

-------------------------------------------
Time: 2020-09-15 13:19:13
-------------------------------------------

-------------------------------------------
Time: 2020-09-15 13:19:14
-------------------------------------------

-------------------------------------------
Time: 2020-09-15 13:19:15
-------------------------------------------

-------------------------------------------
Time: 2020-09-15 13:19:16
-------------------------------------------
('world', 1)
('hello', 1)

-------------------------------------------
Time: 2020-09-15 13:19:17
-------------------------------------------



## 基本概念

### 流上下文

**流上下文(StreamingContext)**是所有Spark Streaming函数的主要入口点，可从SparkContext对象创建。

```python
from pyspark import SparkContext
from pyspark.streaming import StreamingContext

sc = SparkContext(master, appName)
ssc = StreamingContext(sc, 1)
```

定义上下文后，必须执行以下操作。

1. 通过创建输入DStream定义输入源。
2. 通过将转换和输出操作应用于DStream来定义流计算。
3. 开始接收数据并使用进行处理`streamingContext.start()`。
4. 等待使用停止处理（手动或由于任何错误）`streamingContext.awaitTermination()`。
5. 可以使用手动停止处理`streamingContext.stop()`。

**要点**：
- 一旦启动上下文，就无法设置新的流计算或将其添加到该流计算中。
- 上下文一旦停止，就无法重新启动。
- JVM中只能同时激活一个StreamingContext。
- StreamingContext上的stop()也会停止SparkContext。要仅停止的StreamingContext，设置`stop()`的可选参数stopSparkContext为false。
- 只要在创建下一个StreamingContext之前停止（而不停止SparkContext）上一个StreamingContext，即可将SparkContext重用于创建多个StreamingContext。

### 流

**流(DStream)**是Spark Streaming提供的基本抽象，它表示连续的数据流。在内部，DStream由一系列连续的RDD表示，每个RDD都包含特定间隔的数据，如下图：

![streaming-dstream](../images/streaming-dstream.png)

在DStream上执行的任何操作都转换为对基础RDD的操作,如下图：

![streaming-dstream-ops](../images/streaming-dstream-ops.png)

基础的RDD转换由Spark引擎计算。


### 输入流和接收器

输入流(Input DStream)是表示从流源接受输入数据的DStream，在[Hello World](#hello-world)例子中，lines即为输入流，表示从netcat服务器接收的数据流。每个输入流（文件流除外）都与一个**接收器(Receiver)**关联，接收器从源接收数据并将其存储在Spark的内存中进行处理。

Spark Streaming提供了两类内置的流媒体源。
- 基础源：可直接在StreamingContext API中获得的源。如：文件系统和套接字连接
- 高级源：可以通过其他实用程序类如Kafka，Kinesis等获得资源。

流应用程序中可创建过个输入流及接收器，可同时接收多个数据流。

**要点**
- 在本地运行流应用程序时，请勿使用`"local"`或`"local[1]"`作为Master URL。这两种方式均意味着仅一个线程将用于本地运行任务。如果使用了关联接收器（例如套接字，Kafka等）的输入流，则将使用单个线程来运行接收器，而不会留下任何线程来处理接收到的数据。因此，在本地运行时，请始终使用`"local[n]"`作为Master URL，其中n>要运行的接收器数。
- 同理，在集群上运行流应用程序，分配给流应用程序的内核数必须大于接收器数。否则，系统能接收数据，但无法处理它。

#### 基础源

**基于TCP套接字创建流**: ` ssc.socketTextStream(...)`。在例子[Hello World](#hello-world)中已展示。

**基于文件系统创建流**：`streamingContext.textFileStream(dataDirectory)`。读取任何兼容HDFS API(如HDFS、S3、NFS等)的文件系统中的文件来创建DStream。文件系统不需要运行接收器，因此不需要分配任何内核来接受文件数据。若输入为目录，Spark Streaming将监视目录并处理目录中创建的所有文件。

**基于RDD队列创建流**：`streamingContext.queueStream(queueOfRDDs)`。

****

#### 高级源
从Spark 3.0.1开始，Python API中提供了Kafka和Kinesis源。

- Kafka： Spark Streaming 3.0.1与0.10或更高版本的Kafka代理兼容。有关更多详细信息，请参见《[Kafka集成指南](https://spark.apache.org/docs/latest/streaming-kafka-0-10-integration.html)》。
- Kinesis： Spark Streaming 3.0.1与Kinesis Client Library 1.2.1兼容。有关更多详细信息，请参见《[Kinesis集成指南](https://spark.apache.org/docs/latest/streaming-kinesis-integration.html)》。

#### 自定义源
Python API尚不支持。

#### 接收器的可靠性
根据是否会发送反馈，接收器分为

- **可靠接收器**:接受到数据并存储到Spark中后，会向源发送acknowledgement

- **不可靠接收器**:不会向源发送acknowledgement

### 流的转换操作

与RDD类似，流的转换用于修改输入流的数据。下表为常见的一些流的转换。

转换|含义
:--|:--
**map**(*func*)|返回一个*func*计算结果的新DStream
**flatMap**(*func*) | 类似于map，但是每一个输入元素可以被映射为0或多个输出元素（所以*func*应该返回一个序列，而不是单一元素）
**filter**(*func*) | 返回一个挑选*func*计算结果为true的元素的新DStream
**repartition**(*numPartitions*) | 重新分区，必须shuffle，参数是要分多少区
**union**(*otherDStream*) | 返回源DStream和*otherDStream*的并集DStream
**count**() | 返回一个单元素RDD的DStream，统计每个RDD的元素个数
**reduce**(*func*) | 类似count，不过这里指定*func*作为”加“算子
**countByValue**() | 返回（K，V）对的DStream，V为键K在每个RDD中的频率
**reduceByKey**(*func*,\[*numTasks*\]) | 类似countByValue，不过这里指定*func*作为”加“算子
**join**(*otherStream*, \[*numTasks*\]) | 在(K,V)和(K,W)的DStream上调用，返回(K,(V,W))的DStream，其中每个键都有所有成对的元素，相当于内连接(求交集)。
**cogroup**(*otherStream*, \[*numTasks*\]) | 在(K,V)和(K,W)的DStream上调用，返回一个`(K, Seq[V],Seq[W])`类型的DStream。
**transform**(*func*) | 执行任意的RDD-to-RDD的函数
**updateStateByKey**(*func*) | 返回一个更新状态的DStream， 每个键对应的状态通过给定函数*func*、先前状态和当前状态来更新

下面详细讨论下部分常用转换。

#### 有状态转换操作

**updateStateByKey**维持一个状态并连续地根据获得的新信息对状态进行更新。需要定义：
- 状态 - 可以是任意数据类型
- 状态更新函数 - 参数为先前状态和从输入流中获得的新值

每次批处理中，Spark会对所有存在的键值执行更新函数，不管其在批数据中有无数据，如果更新函数返回`None`则该键值对会被消除。

对Hello World的例子进行拓展，记录每个单词在文本数据流中的计数：
```python
def updateFunction(newValues, runningCount):
    if runningCount is None:
        runningCount = 0
    return sum(newValues, runningCount)

runningCounts = pairs.updateStateByKey(updateFunction)
```

#### 无状态转换操作
**transform**对流执行任意RDD-to-RDD的函数，可用来执行DStream API中没有的RDD操作，如将流中每批数据和另一个数据集合并，下面的例子为根据spam信息对数据进行实时清理：
```python
spamInfoRDD = sc.pickleFile(...)  # RDD containing spam information

# join data stream with spam information to do data cleaning
cleanedDStream = wordCounts.transform(lambda rdd: rdd.join(spamInfoRDD).filter(...))
```

#### 窗口操作

Spark Streaming支持窗口计算，允许用户对滑动窗口数据进行转换，如下图：

![streaming-dstream-window](../images/streaming-dstream-window.png)

任何窗口操作都需要指定两个参数：

- windowLength: 窗口长度(上图中windowLength为3)
- slideInterval: 进行窗口操作的间隔(上图中slideInterval为2)

对Hello World的例子进行拓展，实现每10s统计一次最后30s数据的字数
```python
windowedWordCounts = pairs.reduceByKeyAndWindow(lambda x, y: x + y, lambda x, y: x - y, 30, 10)
```

以下为常见的窗口操作

转换 | 含义
:-- | :--
**window**(*windowLength*, *slideInterval*) | 返回基于源DStream的窗口批处理计算的新DStream。
**countByWindow**(*windowLength*, *slideInterval*) | 	返回流中元素的滑动窗口计数。
**reduceByWindow**(*func*, *windowLength*, *slideInterval*) | 返回一个新的单元素流，该流是通过使用*func*算子对流中滑动间隔内的元素聚合而创建的。 该函数应该为一种满足结合律和交换律的二元算子，以便可以并行正确地计算。
**reduceByKeyAndWindow**(*func*, *windowLength*, *slideInterval*, \[*numTasks*\]) | 当在(K, V)对的DStream上调用时，返回一个新的(K, V)对的DStream，其中每个键的值由使用给定的reduce函数*func*对滑动窗口的批数据聚合而得。 注意：默认情况下，它使用Spark的默认并行任务数(本地模式为2，而在集群模式下，此数量由config属性`spark.default.parallelism`确定)进行分组。 您可以传递一个可选的`numTasks`参数来设置不同数量的任务。
**reduceByKeyAndWindow**(*func*, *invfunc*, *windowLength*, *slideInterval*, \[*numTasks*\]) | 上述`reduceByKeyAndWindow()`的更高效的版本，其中，使用前一个窗口的缩减值递增地计算每个窗口的缩减值。 这是通过减少进入滑动窗口的新数据，然后“反减少”离开窗口的旧数据来完成的。 一个示例是在窗口滑动时“增加”和“减少”键的计数。 但是，它仅适用于“可逆归约函数”，即具有相应“逆归约”函数(参数invFunc)的归约函数。 像reduceByKeyAndWindow中一样，reduce任务的数量可以通过可选参数配置。 请注意，必须启用检查点才能使用此操作。
**countByValueAndWindow**(*windowLength*, *slideInterval*, \[*numTasks*\]) | 在(K, V)对的DStream上调用时，返回新的(K, Long)对的DStream，其中每个键的值是其在滑动窗口内的频率。 像reduceByKeyAndWindow中一样，reduce任务的数量可以通过可选参数配置。

#### 合并操作

**流流合并**:
```python
stream1 = ...
stream2 = ...
joinedStream = stream1.join(stream2)
```
与RDD类似，这里的join相当于内连接(求交集)。外连接通过外连接leftOuterJoin，rightOuterJoin和fullOuterJoin支持。

**流和数据集合并**:
```python
dataset = ... # some RDD
windowedStream = stream.window(20)
joinedStream = windowedStream.transform(lambda rdd: rdd.join(dataset))
```
实际上，可以动态更改要合并的数据集。由于提供给`transform`的函数在每个批处理间隔中都会进行评估，因此使用的是数据集引用所指向的实时数据集。

### 流的输出操作

流的输出操作用于将流的数据推送到外部系统如数据库或文件系统。由于输出操作实际上允许外部系统使用转换后的数据，因此它们会触发所有DStream转换的实际执行（类似于RDD的操作）。 当前，定义了以下输出操作：

输出操作 | 含义
:--|:--
**print**() | 在运行流应用程序的驱动程序节点上，打印DStream中每批数据的前十个元素。（Python API为`pprint()`）
**saveAsTextFiles**(*prefix*, \[*suffix*\]) | 将此DStream的内容另存为文本文件。每个批处理间隔的文件名都是基于前缀和后缀“ prefix-TIME_IN_MS\[.*suffix*\]”生成的。(Python API不支持)
**saveAsObjectFiles**(*prefix*, \[*suffix*\]) | 将此DStream的内容另存为序列化Java对象的SequenceFiles。每个批处理间隔的文件名都是基于前缀和后缀“ prefix-TIME_IN_MS\[.*suffix*\]”生成的。(Python API不支持)
**saveAsHadoopFiles**(*prefix*, \[*suffix*\]) | 将此DStream的内容另存为Hadoop文件。每个批处理间隔的文件名都是基于前缀和后缀“ prefix-TIME_IN_MS\[.*suffix*\]”生成的。
**foreachRDD**(*func*) | 最通用的输出运算符，将函数*func*应用于从流生成的每个RDD。 此函数应将每个RDD中的数据推送到外部系统，例如将RDD保存到文件或通过网络将其写入数据库。 请注意，函数*func*在运行流应用程序的驱动程序进程中执行，并且通常在其中具有RDD操作，这将强制计算流RDD。

#### foreachRDD的设计模式

`dstream.foreachRDD`是功能强大的原语，它允许将数据发送到外部系统。但是，重要的是要了解如何正确有效地使用此原语，应避免如下一些常见错误。

通常，将数据写入外部系统需要创建一个连接对象（例如，与远程服务器的TCP连接），然后使用该对象将数据发送到远程系统。为此，开发人员可能会无意间尝试在Spark驱动程序中创建连接对象，然后尝试在Spark辅助程序中使用该对象以将记录保存在RDD中。 例如

```python
def sendRecord(rdd):
    connection = createNewConnection()  # executed at the driver
    rdd.foreach(lambda record: connection.send(record))
    connection.close()

dstream.foreachRDD(sendRecord)
```

这是不正确的，因为这要求将连接对象序列化并从驱动程序发送给工作程序。 这样的连接对象很少能在机器之间转移。 此错误可能表现为序列化错误（连接对象不可序列化），初始化错误（连接对象需要在工作程序中初始化）等。正确的解决方案是在工作程序中创建连接对象。

但是，这可能会导致另一个常见错误-为每个记录创建一个新的连接。 例如，

```python
def sendRecord(record):
    connection = createNewConnection()
    connection.send(record)
    connection.close()

dstream.foreachRDD(lambda rdd: rdd.foreach(sendRecord))
```

通常，创建连接对象会浪费时间和资源。 因此，为每个记录创建和销毁连接对象会导致不必要的高开销，并且会大大降低系统的整体吞吐量。 更好的解决方案是使用rdd.foreachPartition-创建一个连接对象，并使用该连接发送RDD分区中的所有记录。

```python
def sendPartition(iter):
    connection = createNewConnection()
    for record in iter:
        connection.send(record)
    connection.close()

dstream.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))
```

这将分摊许多记录上的连接创建开销。

最后，可以通过在多个RDD /批次之间重用连接对象来进一步优化。 与将多个批次的RDD推送到外部系统时可以重用的连接对象相比，它可以维护一个静态的连接对象池，从而进一步减少了开销。

```python
def sendPartition(iter):
    # ConnectionPool is a static, lazily initialized pool of connections
    connection = ConnectionPool.getConnection()
    for record in iter:
        connection.send(record)
    # return to the pool for future reuse
    ConnectionPool.returnConnection(connection)

dstream.foreachRDD(lambda rdd: rdd.foreachPartition(sendPartition))
```

请注意，应按需延迟创建池中的连接，如果一段时间不使用，则超时。 这样可以最有效地将数据发送到外部系统。

**其他注意点**：

- DStream由输出操作延迟执行，就像RDD由RDD操作延迟执行一样。 具体来说，DStream输出操作内部的RDD动作会强制处理接收到的数据。 因此，如果您的应用程序没有任何输出操作，或者内部没有任何RDD操作，则具有类似`dstream.foreachRDD()`的输出操作，则将不会执行任何操作。 系统将仅接收数据并将其丢弃。
- 默认情况下，输出操作一次执行一次。 它们按照在应用程序中定义的顺序执行。

### DataFrame和SQL操作

### MLlib操作

### 缓存与持久化

与RDD相似，DStreams还允许开发人员将流的数据持久存储在内存中。也就是说，在DStream上使用`persist()`方法将自动将该DStream的每个RDD持久存储在内存中。如果DStream中的数据将被多次计算（例如，对同一数据进行多次操作），这将很有用。 对于基于窗口的操作（如`reduceByWindow`和`reduceByKeyAndWindow`）以及基于状态的操作（如`updateStateByKey`），会进行隐式地持久化，即由基于窗口的操作生成的DStream会自动保存在内存中，而无需开发人员调用`persist()`。

对于通过网络接收数据的输入流（例如Kafka，套接字等），默认的持久性级别设置为将数据复制到两个节点以实现容错。

请注意，与RDD不同，DStream的默认持久性级别为将数据序列化在内存中。

### 检查点

流式应用程序必须24/7全天候运行，因此必须对与应用程序逻辑无关的故障（例如，系统故障，JVM崩溃等）具有弹性。为此，Spark Streaming需要将足够的信息检查点指向容错存储系统，以便可以从故障中恢复。检查点有两种类型的数据。

- 元数据检查点 - 将定义流计算的信息保存到HDFS等容错存储中。这用于从运行流应用程序的驱动程序的节点的故障中恢复。元数据包括：
    - 配置 - 用于创建流应用程序的配置。
    - DStream操作 - 定义流应用程序的DStream操作集。
    - 未完成批次 - 作业排队但尚未完成的批次。
- 数据检查点 - 将生成的RDD保存到可靠的存储中。在一些有状态转换中，这需要跨多个批次合并数据，这是必需的。在此类转换中，生成的RDD依赖于先前批次的RDD，这导致依赖项链的长度随时间不断增加。为了避免这种恢复时间无限制的增加（与依赖关系链成比例），定期将有状态转换的中间RDD检查点指向可靠的存储（例如HDFS），以切断依赖关系链。

总而言之，从驱动程序故障中恢复时，主要需要元数据检查点，而如果使用有状态转换，则即使是基本功能，也需要数据或RDD检查点。

#### 何时需要启用检查点

必须为具有以下任一要求的应用程序启用检查点：

- 使用了有状态的转换操作-如果在应用程序中使用了updateStateByKey或reduceByKeyAndWindow（具有逆函数），则必须提供检查点目录以允许定期进行RDD检查点。
- 从运行应用程序的驱动程序故障中恢复-元数据检查点用于恢复进度信息。

注意，没有前述状态转换的简单流应用程序可以在不启用检查点的情况下运行。在这种情况下，从驱动程序故障中恢复也将是部分的（某些丢失但未处理的数据可能会丢失）。这通常是可以接受的，并且许多Spark Streaming应用程序都以这种方式运行。预计将来会改善对非Hadoop环境的支持。

#### 如何配置检查点

可以通过在容错、可靠的文件系统（例如HDFS，S3等）中设置目录来启用检查点，将检查点信息保存到该目录中。这是通过使用`streamingContext.checkpoint(checkpointDirectory)`完成的。这将允许使用前面提到的有状态转换。此外，如果要使应用程序从驱动程序故障中恢复，则应重写流应用程序以具有以下行为。

- 程序首次启动时，它将创建一个新的StreamingContext，设置所有流，然后调用start()。
- 失败后重新启动程序时，它将根据检查点目录中的检查点数据重新创建StreamingContext。

通过使用`StreamingContext.getOrCreate`可简单创建检查点，如下：

```python
# Function to create and setup a new StreamingContext
def functionToCreateContext():
    sc = SparkContext(...)  # new context
    ssc = StreamingContext(...)
    lines = ssc.socketTextStream(...)  # create DStreams
    ...
    ssc.checkpoint(checkpointDirectory)  # set checkpoint directory
    return ssc

# Get StreamingContext from checkpoint data or create a new one
context = StreamingContext.getOrCreate(checkpointDirectory, functionToCreateContext)

# Do additional setup on context that needs to be done,
# irrespective of whether it is being started or restarted
context. ...

# Start the context
context.start()
context.awaitTermination()
```

如果存在`checkpointDirectory`，则将根据检查点数据重新创建上下文。如果该目录不存在（即首次运行），则将调用函数functionToCreateContext来创建新的上下文并设置DStreams。请参阅Python示例[recoveryable_network_wordcount.py](https://github.com/apache/spark/blob/master/examples/src/main/python/streaming/recoverable_network_wordcount.py)。本示例将网络数据的字数附加到文件中。

您还可以根据检查点数据显式创建`StreamingContext`，并使用`StreamingContext.getOrCreate(checkpointDirectory, None)`开始计算。

除了使用`getOrCreate`之外，还需要确保驱动程序进程在发生故障时自动重新启动。这只能通过用于运行应用程序的部署基础结构来完成。见[部署流应用](#部署流应用)

请注意，RDD的检查点会导致保存到可靠存储的成本。这可能会导致RDD获得检查点的那些批次的处理时间增加。因此，需要仔细设置检查点的间隔。在小批量（例如1秒）时，每批检查点可能会大大降低操作吞吐量。相反，检查点太少会导致随着运算链和任务规模增加会产生不利影响。对于需要RDD检查点的有状态转换，默认间隔为批处理间隔的倍数，至少应为10秒。可以使用`dstream.checkpoint(checkpointInterval)`进行设置。通常，DStream的5-10个滑动间隔的检查点间隔是一个较好的设置。

### 累加器、广播变量和检查点

无法从Spark Streaming中的检查点恢复累加器和广播变量。如果启用检查点并同时使用“累加器”或“广播”变量，则必须为“累加器”和“广播”变量创建延迟实例化的单例实例，以便在驱动程序因故障而重新启动后可以重新实例化它们。如下所示：

```python
def getWordBlacklist(sparkContext):
    if ("wordBlacklist" not in globals()):
        globals()["wordBlacklist"] = sparkContext.broadcast(["a", "b", "c"])
    return globals()["wordBlacklist"]

def getDroppedWordsCounter(sparkContext):
    if ("droppedWordsCounter" not in globals()):
        globals()["droppedWordsCounter"] = sparkContext.accumulator(0)
    return globals()["droppedWordsCounter"]

def echo(time, rdd):
    # Get or register the blacklist Broadcast
    blacklist = getWordBlacklist(rdd.context)
    # Get or register the droppedWordsCounter Accumulator
    droppedWordsCounter = getDroppedWordsCounter(rdd.context)

    # Use blacklist to drop words and use droppedWordsCounter to count them
    def filterFunc(wordCount):
        if wordCount[0] in blacklist.value:
            droppedWordsCounter.add(wordCount[1])
            False
        else:
            True

    counts = "Counts at time %s %s" % (time, rdd.filter(filterFunc).collect())

wordCounts.foreachRDD(echo)
```

### 部署流应用

本部分讨论了部署Spark Streaming应用程序的步骤。

#### 要求

要运行Spark Streaming应用程序，需要具备以下条件。
- 集群具备集群管理器 - 这是任何Spark应用程序的一般要求，并且在[部署指南](https://spark.apache.org/docs/latest/cluster-overview.html)中进行了详细讨论。
- 将应用程序JAR打包 - 您必须将流式应用程序编译为JAR。如果使用[spark-submit](https://spark.apache.org/docs/latest/submitting-applications.html)来启动应用程序，则无需在JAR中提供Spark和Spark Streaming。但是，如果您的应用程序使用高级源（例如Kafka），则必须将它们链接到的额外工件及其依赖项打包在用于部署应用程序的JAR中。例如，使用KafkaUtils的应用程序必须在应用程序JAR中包含`spark-streaming-kafka-0-10_2.12`及其所有传递依赖项。
- 为执行者配置足够的内存 - 由于必须将接收到的数据存储在内存中，因此必须为执行者配置足够的内存来保存接收到的数据。请注意，如果您要执行10分钟的窗口操作，则系统必须在内存中至少保留最后10分钟的数据。因此，应用程序的内存要求取决于应用程序中使用的操作。
- 配置检查点 - 如果流应用程序需要它，则必须将与Hadoop API兼容的容错存储中的目录（例如HDFS，S3等）配置为检查点目录，并且以这样的方式编写流应用程序以用于故障恢复。
- 配置应用程序驱动程序的自动重新启动 - 若要从驱动程序故障中自动恢复，用于运行流式应用程序的部署基础结构必须监视驱动程序进程，并在驱动程序失败时重新启动。不同的集群管理器具有不同的工具来实现这一目标。
    - Spark Standalone - 可以提交Spark应用程序驱动程序以在Spark Standalone集群中运行（请参阅[集群部署模式](https://spark.apache.org/docs/latest/spark-standalone.html#launching-spark-applications)），即，应用程序驱动程序本身在工作程序节点之一上运行。此外，可以指示独立群集管理器监督驱动程序，并在驱动程序由于非零退出代码或由于运行该驱动程序的节点故障而失败时重新启动它。有关更多详细信息，请参见Spark Standalone指南中的集群模式和监督。
    - YARN - Yarn支持自动重启应用程序的类似机制。请参阅YARN文档以获取更多详细信息。
    - Mesos - [marathon](https://github.com/mesosphere/marathon)已经被Mesos用来实现这一目标。
- 配置预写日志 - Spark自1.2起，引入了预写日志以实现强大的容错保证。如果启用，则将从接收器接收的所有数据写入配置检查点目录中的预写日志。这样可以防止驱动程序恢复时丢失数据，从而确保零数据丢失（见[容错语义](#容错语义)）。可以通过将配置参数`spark.streaming.receiver.writeAheadLog.enable`设置为true来启用此功能。但是，这些更强的语义可能以单个接收器的接收吞吐量为代价。可以通过并行运行更多接收器以提高总吞吐量来纠正此问题。另外，由于启用了预写日志，因此建议禁用Spark中接收数据的复制，因为该日志已经存储在复制的存储系统中。这可以通过将输入流的存储级别设置为`StorageLevel.MEMORY_AND_DISK_SER`来完成。在将S3（或任何不支持刷新的文件系统）用于预写日志时，请记住启用`spark.streaming.driver.writeAheadLog.closeFileAfterWrite`和`spark.streaming.receiver.writeAheadLog.closeFileAfterWrite`。有关更多详细信息，请参见[Spark Streaming配置](https://spark.apache.org/docs/latest/configuration.html#spark-streaming)。请注意，启用I/O加密后，Spark不会加密写入预写日志的数据。如果需要对预写日志数据进行加密，则应将其存储在本身支持加密的文件系统中。
- 设置最大接收速率-如果群集资源不足以使流应用程序能够以最快的速度处理数据，则可以通过设置记录/秒的最大速率限制来限制接收器的速率。请参阅配置参数`spark.streaming.receiver.maxRate`（用于接收器）和`spark.streaming.kafka.maxRatePerPartition`（用于Direct Kafka方法）。Spark 1.5引入了一个称为背压的功能，该功能消除了设置此速率限制的需要，因为Spark Streaming会自动计算出速率限制，并在处理条件发生变化时动态调整它们。可以通过将配置参数`spark.streaming.backpressure.enabled`设置为true来启用此背压。

#### 升级应用代码

如果需要使用新的应用程序代码升级正在运行的Spark Streaming应用程序，则有两种可能的机制。

- 升级后的Spark Streaming应用程序将启动并与现有应用程序并行运行。一旦新的（接收与旧的数据相同）的数据被预热并准备好进入黄金时段，就可以关闭旧的数据。请注意，对于支持将数据发送到两个目标的数据源（即较早的应用程序和升级的应用程序），可以这样做。
- 正常关闭现有应用程序，以确保在关闭之前可以完全处理已接收的数据。然后可以启动升级的应用程序，它将从较早的应用程序停止的同一点开始进行处理。请注意，只能使用支持源端缓冲的输入源（例如Kafka）来完成此操作，因为在上一个应用程序关闭且升级的应用程序尚未启动时，需要缓冲数据。并且无法从升级前代码的较早检查点信息重新启动。检查点信息本质上包含序列化的Scala/Java/Python对象，尝试使用经过修改的新类反序列化对象可能会导致错误。在这种情况下，请使用其他检查点目录启动升级的应用程序，或者删除先前的检查点目录。

### 监视流应用

除了Spark的[监视功能](https://spark.apache.org/docs/latest/monitoring.html)外，Spark Streaming还具有其他特定功能。使用StreamingContext时，[Spark Web UI](https://spark.apache.org/docs/latest/monitoring.html#web-interfaces)会显示一个附加的Streaming选项卡，其中显示有关正在运行的接收器（接收器是否处于活动状态，接收到的记录数，接收器错误等）和完成的批处理（批处理时间，排队延迟等）的统计信息。）。这可用于监视流应用程序的进度。

Web UI中的以下两个特别重要的指标：

- 处理时间-处理每批数据的时间。
- 计划延迟-批次在队列中等待之前的批次完成处理的时间。

如果批处理时间始终大于批处理时间间隔和/或排队延迟持续增加，则表明系统无法像生成批处理一样快处理批处理，并且落后于此。在这种情况下，请考虑减少批处理时间。

还可以使用StreamingListener界面监视Spark Streaming程序的进度，该界面可让您获取接收器状态和处理时间。请注意，这是一个开发人员API，将来可能会得到改进。

## 性能调优

要在集群上的Spark Streaming应用程序中获得最佳性能，需要进行一些调整。本节说明了可以调整以提高应用程序性能的许多参数和配置。从高层次上讲，需要考虑两件事：
- 通过有效地使用集群资源来减少每批数据的处理时间。
- 设置正确的批处理大小，以便可以在接收到批处理数据后尽快对其进行处理（也就是说，数据处理与数据获取保持同步）。

### 减少批处理时间

在Spark中可以进行许多优化，以最大程度地减少每批的处理时间。这些已在《[调优指南](https://spark.apache.org/docs/latest/tuning.html)》中详细讨论。 本节重点介绍一些最重要的内容。

#### 数据接收中的并行度

通过网络（例如Kafka，套接字等）接收数据需要将数据反序列化并存储在Spark中。 如果数据接收成为系统的瓶颈，请考虑并行化数据接收。 请注意，每个输入DStream都会创建一个接收器（在工作计算机上运行），该接收器接收单个数据流。 因此，可以通过创建多个输入DStream并将其配置为从源接收数据流的不同分区来实现接收多个数据流。 例如，可以将接收两个主题数据的单个Kafka输入DStream拆分为两个Kafka输入流，每个输入流仅接收一个主题。 这将运行两个接收器，从而允许并行接收数据，从而提高了总体吞吐量。 这些多个DStream可以结合在一起以创建单个DStream。 然后，可以将应用于单个输入DStream的转换应用于统一流。 这样做如下。

```python


```

应该考虑的另一个参数是接收器的块间隔，该间隔由配置参数spark.streaming.blockInterval确定。 对于大多数接收器而言，接收到的数据在存储在Spark内存中之前会合并为数据块。 每批中的块数确定了将在类似地图的转换中用于处理接收到的数据的任务数。 每批每个接收者的任务数大约为（批处理间隔/块间隔）。 例如，200 ms的块间隔将每2秒批处理创建10个任务。 如果任务数太少（即少于每台计算机的核心数），那么它将效率低下，因为将不会使用所有可用的核心来处理数据。 要增加给定批处理间隔的任务数，请减小阻止间隔。 但是，建议的块间隔最小值约为50毫秒，在此之下，任务启动开销可能是个问题。

使用多个输入流/接收器接收数据的一种替代方法是显式重新划分输入数据流（使用inputStream.repartition（<分区数>））。 在进一步处理之前，这会将接收到的数据批分布在群集中指定数量的计算机上。

对于直接流，请参阅[Spark Streaming + Kafka集成指南](https://spark.apache.org/docs/latest/streaming-kafka-0-10-integration.html)

#### 数据处理中的并行度

如果在计算的任何阶段使用的并行任务数量不够高，则群集资源可能无法得到充分利用。 例如，对于诸如reduceByKey和reduceByKeyAndWindow之类的分布式reduce操作，并行任务的默认数目由spark.default.parallelism配置属性控制。 您可以将并行性级别作为参数传递（请参见PairDStreamFunctions文档），或设置spark.default.parallelism配置属性以更改默认值。

#### 数据序列化

可以通过调整序列化格式来减少数据序列化的开销。 在流传输的情况下，有两种类型的数据正在序列化。
- 输入数据：默认情况下，通过Receiver接收的输入数据通过StorageLevel.MEMORY_AND_DISK_SER_2存储在执行程序的内存中。 也就是说，数据被序列化为字节以减少GC开销，并被复制以容忍执行器故障。 同样，数据首先保存在内存中，并且仅在内存不足以容纳流计算所需的所有输入数据时才溢出到磁盘。 显然，这种序列化会产生开销-接收者必须对接收到的数据进行反序列化，然后使用Spark的序列化格式对其进行重新序列化。
- 流操作生成的持久RDD：流计算生成的RDD可能会保留在内存中。 例如，窗口操作会将数据保留在内存中，因为它们将被多次处理。 但是，与Spark Core默认的StorageLevel.MEMORY_ONLY不同，默认情况下，由流式计算生成的持久性RDD与StorageLevel.MEMORY_ONLY_SER（即已序列化）保持一致，以最大程度地减少GC开销。

在这两种情况下，使用Kryo序列化都可以减少CPU和内存的开销。 有关更多详细信息，请参见《 Spark Tuning Guide》。 对于Kryo，请考虑注册自定义类，并禁用对象引用跟踪（请参阅《配置指南》中与Kryo相关的配置）。

在流应用程序需要保留的数据量不大的特定情况下，将数据（两种类型）保留为反序列化对象是可行的，而不会产生过多的GC开销。 例如，如果您使用的是几秒钟的批处理间隔并且没有窗口操作，那么您可以尝试通过显式设置存储级别来禁用持久化数据中的序列化。 这将减少由于序列化导致的CPU开销，从而可能在没有太多GC开销的情况下提高性能。

#### 任务启动开销

如果每秒启动的任务数量很高（例如，每秒50个或更多），那么向从服务器发送任务的开销可能会很大，并且将难以实现亚秒级的延迟。 可以通过以下更改来减少开销：

- 执行模式：在独立模式或粗粒度Mesos模式下运行Spark可以比细粒度Mesos模式缩短任务启动时间。 有关更多详细信息，请参阅“在Mesos上运行”指南。

这些更改可以将批处理时间减少100毫秒，从而使亚秒级的批处理大小可行。

### 设置正确的批间隔

为了使在群集上运行的Spark Streaming应用程序稳定，系统应能够尽快处理接收到的数据。 换句话说，应尽快处理一批数据。 可以通过监视流式Web UI中的处理时间来发现这是否适用于应用程序，其中批处理时间应小于批处理间隔。
    
根据流计算的性质，所使用的批处理间隔可能会对数据速率产生重大影响，该速率可以由应用程序在固定的一组群集资源上维持。 例如，让我们考虑前面的WordCountNetwork示例。 对于特定的数据速率，系统可能能够跟上每2秒（即2秒的批处理间隔）报告字数，但不是每500毫秒。 因此，需要设置批次间隔，以便可以维持生产中的预期数据速率。

找出适合您的应用程序的正确批处理大小的一种好方法是使用保守的批处理间隔（例如5-10秒）和低数据速率进行测试。 要验证系统是否能够跟上数据速率，您可以检查每个已处理批次经历的端到端延迟的值（可以在Spark驱动程序log4j日志中查找“ Total delay”，也可以使用 StreamingListener接口）。 如果延迟保持与批次大小相当，则系统是稳定的。 否则，如果延迟持续增加，则意味着系统无法跟上，因此不稳定。 一旦有了稳定配置的想法，就可以尝试提高数据速率和/或减小批处理大小。 注意，只要延迟减小回到低值（即小于批大小），由于临时数据速率增加而引起的延迟的瞬时增加可能是好的。

### 内存调优

《调整指南》中详细讨论了调整Spark应用程序的内存使用情况和GC行为。 强烈建议您阅读。 在本节中，我们将专门在Spark Streaming应用程序的上下文中讨论一些调整参数。
    
Spark Streaming应用程序所需的群集内存量在很大程度上取决于所使用的转换类型。 例如，如果要对最后10分钟的数据使用窗口操作，则群集应具有足够的内存以在内存中保存价值10分钟的数据。 或者，如果要对大量键使用updateStateByKey，则所需的内存将很大。 相反，如果您想执行一个简单的map-filter-store操作，则所需的内存将很少。

通常，由于通过接收器接收的数据存储在StorageLevel.MEMORY_AND_DISK_SER_2中，因此无法容纳在内存中的数据将溢出到磁盘上。 这可能会降低流应用程序的性能，因此建议您提供流应用程序所需的足够内存。 最好尝试以小规模查看内存使用情况并据此进行估计。

内存调整的另一个方面是垃圾回收。 对于需要低延迟的流应用程序，不希望由于JVM垃圾收集而导致较大的停顿。
    
有一些参数可以帮助您调整内存使用情况和GC开销：
- DStreams的持久性级别：如前面在“数据序列化”部分所述，默认情况下，输入数据和RDD作为序列化字节保留。 与反序列化的持久性相比，这减少了内存使用和GC开销。 启用Kryo序列化可进一步减少序列化的大小和内存使用量。 通过压缩可以进一步减少内存使用（请参见Spark配置spark.rdd.compress），而这会占用CPU时间。
- 清除旧数据：默认情况下，将自动清除DStream转换生成的所有输入数据和持久的RDD。  Spark Streaming根据使用的转换来决定何时清除数据。 例如，如果您使用10分钟的窗口操作，那么Spark Streaming将保留最后10分钟的数据，并主动丢弃较旧的数据。 通过设置streamingContext.remember，可以将数据保留更长的时间（例如，以交互方式查询较旧的数据）。
- CMS垃圾收集器：强烈建议使用并发标记和清除GC，以使与GC相关的暂停时间始终保持较低。 尽管已知并发GC会降低系统的整体处理吞吐量，但仍建议使用它来实现更一致的批处理时间。 确保在驱动程序（使用spark-submit中使用--driver-java-options）和执行程序（使用Spark配置spark.executor.extraJavaOptions）上都设置了CMS GC。
- 其他技巧：为了进一步减少GC开销，请尝试以下更多技巧。
    - 使用OFF_HEAP存储级别持久保留RDD。 请参阅《 Spark编程指南》中的更多详细信息。
    - 使用更多具有较小堆大小的执行程序。 这将减少每个JVM堆中的GC压力。
    
**要点**:
- DStream与单个接收器关联。 为了获得读取并行性，需要创建多个接收器，即多个DStream。 接收器在执行器中运行。 它占据了一个核心。 预订接收器插槽后，请确保有足够的内核可用于处理，即spark.cores.max应考虑接收器插槽。 接收者以循环方式分配给执行者。
- 当从流源接收数据时，接收器会创建数据块。 每blockInterval毫秒生成一个新的数据块。 在batchInterval期间创建了N个数据块，其中N = batchInterval / blockInterval。 这些块由当前执行器的BlockManager分发给其他执行器的块管理器。 之后，驱动程序上运行的网络输入跟踪器将被告知有关块的位置，以进行进一步处理。
- 在驱动程序上为在batchInterval期间创建的块创建了RDD。 在batchInterval期间生成的块是RDD的分区。 每个分区都是一个任务。  blockInterval == batchinterval意味着将创建一个分区，并且可能在本地对其进行处理。
- 块上的映射任务在执行器中进行处理（一个执行器接收该块，另一个执行器复制该块），该执行器具有与块间隔无关的块，除非执行非本地调度。除非间隔时间越长，块间隔越大，意味着块越大。 较高的spark.locality.wait值会增加在本地节点上处理块的机会。 需要在这两个参数之间找到平衡，以确保较大的块在本地处理。
- 您可以通过调用inputDstream.repartition（n）来定义分区数，而不是依赖于batchInterval和blockInterval。 这会随机重新随机排列RDD中的数据以创建n个分区。 是的，以获得更大的并行度。 尽管以洗牌为代价。  RDD的处理由驾驶员的工作计划者安排为工作。 在给定的时间点，只有一项作业处于活动状态。 因此，如果一个作业正在执行，则其他作业将排队。
- 如果您有两个dstream，将形成两个RDD，并且将创建两个作业，这些作业将一个接一个地调度。 为避免这种情况，可以合并两个dstream。 这将确保为dstream的两个RDD形成单个unionRDD。 然后将此unionRDD视为一项工作。 但是，RDD的分区不受影响。
- 如果批处理时间超过batchinterval，那么显然接收方的内存将开始填满，并最终引发异常（最有可能是BlockNotFoundException）。 当前，无法暂停接收器。 使用SparkConf配置spark.streaming.receiver.maxRate，可以限制接收器的速率。

## 容错语义

在本节中，我们将讨论发生故障时Spark Streaming应用程序的行为。

### 背景

要了解Spark Streaming提供的语义，让我们记住Spark的RDD的基本容错语义。

1. RDD是一个不变的，确定性可重新计算的分布式数据集。 每个RDD都会记住在容错输入数据集上用于创建它的确定性操作的沿袭。
2. 如果由于工作节点故障而导致RDD的任何分区丢失，则可以使用操作沿袭从原始容错数据集中重新计算该分区。
3. 假设所有RDD转换都是确定性的，则最终转换后的RDD中的数据将始终相同，而不管Spark集群中的故障如何。

Spark在容错文件系统（例如HDFS或S3）中的数据上运行。 因此，从容错数据生成的所有RDD也是容错的。 但是，Spark Streaming并非如此，因为大多数情况下是通过网络接收数据的（使用fileStream时除外）。 为了对所有生成的RDD实现相同的容错属性，将接收到的数据复制到集群中工作节点中的多个Spark执行程序中（默认复制因子为2）。 这导致系统中的两种数据在发生故障时需要恢复：

1. 接收和复制的数据-该数据在单个工作程序节点发生故障时仍然存在，因为它的副本存在于其他节点之一上。
2. 已接收但已缓冲数据以进行复制-由于不进行复制，因此恢复此数据的唯一方法是从源重新获取数据。

此外，我们应该关注两种故障：

1. 工作节点的故障-运行执行程序的任何工作节点都可能发生故障，并且这些节点上的所有内存中数据都将丢失。 如果有任何接收器在发生故障的节点上运行，则其缓冲的数据将丢失。
2. 驱动程序节点发生故障-如果运行Spark Streaming应用程序的驱动程序节点发生故障，则显然SparkContext会丢失，并且所有执行程序及其内存中的数据也会丢失。

有了这些基本知识，让我们了解Spark Streaming的容错语义。

### 定义

流系统的语义通常是根据系统可以处理每个记录多少次来捕获的。 系统可以在所有可能的操作条件（尽管有故障等）下提供三种保证：

1. 最多一次：每条记录要么处理一次，要么根本不处理。
2. 至少一次：每条记录将被处理一次或多次。 它比最多一次强，因为它确保不会丢失任何数据。 但是可能会有重复。
3. 恰好一次：每条记录将恰好处理一次-不会丢失任何数据，也不会多次处理任何数据。 这显然是三者中最强有力的保证。

### 基础语义

概括地说，在任何流处理系统中，处理数据都需要三个步骤。

1. 接收数据：使用接收器或其他方式从源接收数据。
2. 转换数据：使用DStream和RDD转换对接收到的数据进行转换。
3. 推送数据：将最终转换后的数据推送到外部系统，例如文件系统，数据库，仪表板等。
    
如果流应用程序必须获得端到端的精确一次保证，那么每个步骤都必须提供精确一次保证。 也就是说，每个记录必须被准确接收一次，被转换一次，并被推送到下游系统一次。 让我们了解Spark Streaming上下文中这些步骤的语义。

1. 接收数据：不同的输入源提供不同的保证。 下一部分将对此进行详细讨论。
2. 转换数据：由于RDD提供的保证，所有接收到的数据将只处理一次。 即使出现故障，只要可以访问接收到的输入数据，最终转换后的RDD将始终具有相同的内容。
3. 推送数据：默认情况下，输出操作会确保至少一次语义，因为它取决于输出操作的类型（是否为幂等）和下游系统的语义（是否支持事务）。 但是用户可以实现自己的事务处理机制来实现一次语义。 本节稍后将对此进行详细讨论。

### 接收数据的语义

不同的输入源提供不同的保证，范围从至少一次到恰好一次。 阅读更多详细信息。

#### 对于文件

如果所有输入数据已经存在于诸如HDFS之类的容错文件系统中，则Spark Streaming始终可以从任何故障中恢复并处理所有数据。 这提供了一次精确的语义，这意味着无论发生什么故障，所有数据都会被精确处理一次。

#### 对于基于接收器的源

对于基于接收方的输入源，容错语义取决于故障情况和接收方的类型。 如前所述，接收器有两种类型：

- 可靠接收器-这些接收器仅在确保已复制接收到的数据之后才确认可靠源。 如果这样的接收器发生故障，则源将不会收到对缓冲（未复制）数据的确认。 因此，如果重新启动接收器，则源将重新发送数据，并且不会由于失败而丢失任何数据。
- 不可靠的接收器-此类接收器不发送确认，因此当由于工作程序或驱动程序故障而失败时，可能会丢失数据。

根据所使用的接收器类型，我们可以实现以下语义。 如果工作节点发生故障，那么可靠的接收器不会造成数据丢失。 如果接收器不可靠，则接收到但未复制的数据可能会丢失。 如果驱动程序节点发生故障，则除了这些丢失之外，所有已接收并复制到内存中的过去数据都将丢失。 这将影响有状态转换的结果。
    
为了避免丢失过去收到的数据，Spark 1.2引入了预写日志，该日志将收到的数据保存到容错存储中。 使用启用预写日志和可靠的接收器，数据丢失为零。 就语义而言，它至少提供了一次保证。
    
下表总结了失败时的语义：

部署方案 | Worker失败 | Driver失败
--|--|--
Spark 1.1或更早版本<br>不带预写日志的Spark 1.2或更高版本 | 接收器不可靠导致缓冲数据丢失<br>可靠的接收器实现零数据丢失<br>至少一次语义 | 接收器不可靠导致缓冲数据丢失<br>过去的数据丢失了所有接收者<br>未定义的语义
带预写日志的Spark 1.2或更高版本 | 可靠的接收器实现零数据丢失<br>至少一次语义 | 可靠的接收器和文件可实现零数据丢失<br>至少一次语义

#### 对于Kafka Direct API

在Spark 1.3中，我们引入了新的Kafka Direct API，它可以确保Spark Streaming一次接收所有Kafka数据。 同时，如果您执行一次精确的输出操作，则可以实现端到端的一次精确保证。《 Kafka集成指南》中进一步讨论了这种方法。

### 语义的输出操作

输出操作（如foreachRDD）至少具有一次语义，也就是说，在工作程序失败的情况下，转换后的数据可能多次写入外部实体。 尽管这对于使用saveAs *** Files操作保存到文件系统是可以接受的（因为文件将简单地被相同的数据覆盖），但可能需要付出额外的努力才能实现一次精确的语义。 有两种方法。

- 幂等更新：多次尝试总是写入相同的数据。 例如，saveAs *** Files始终将相同的数据写入生成的文件。
- 事务性更新：所有更新都是以事务方式进行的，因此原子更新仅进行一次。 一种做到这一点的方法如下。
    - 使用批处理时间（在foreachRDD中可用）和RDD的分区索引来创建标识符。 该标识符唯一地标识流应用程序中的Blob数据。
    - 使用标识符以事务方式（即，原子地一次）更新此blob的外部系统。 也就是说，如果尚未提交标识符，则自动提交分区数据和标识符。 否则，如果已经提交，则跳过更新。

<head> 
    <script defer src="https://use.fontawesome.com/releases/v5.0.13/js/all.js"></script> 
    <script defer src="https://use.fontawesome.com/releases/v5.0.13/js/v4-shims.js"></script> 
</head> 
<link rel="stylesheet" href="https://use.fontawesome.com/releases/v5.0.13/css/all.css">